In [1]:
import os

import pandas as pd
import polars as pl

import numpy as np

import kaggle_evaluation.jane_street_inference_server

The evaluation API requires that you set up a server which will respond to inference requests. We have already defined the server; you just need write the predict function. When we evaluate your submission on the hidden test set the client defined in `jane_street_gateway` will run in a different container with direct access to the hidden test set and hand off the data timestep by timestep.



Your code will always have access to the published copies of the files.

In [2]:
lags_ : pl.DataFrame | None = None


# Replace this function with your inference code.
# You can return either a Pandas or Polars dataframe, though Polars is recommended.
# Each batch of predictions (except the very first) must be returned within 1 minute of the batch features being provided.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # All the responders from the previous day are passed in at time_id == 0. We save them in a global variable for access at every time_id.
    # Use them as extra features, if you like.

    global lags_
    if lags is not None:
        lags_ = lags

    test_df = test.to_pandas()
    lags_df = lags_.to_pandas()

#     test = test.to_pandas()
#     lags_ = lags_.to_pandas()
    if isinstance(test, pl.DataFrame):
        test.to_pandas()
    if isinstance(lags_, pl.DataFrame):
        lags_.to_pandas()
        
    X_test = pd.merge(test_df, lags_df, on=['time_id', 'symbol_id'], how='left', suffixes = ("","_drop"))
#     print(X_test)

#     print(type(test))
#     print(type(lags_))
#     data = pd.merge(left=test, right=lags_, how='left', left_on=['time_id', 'symbol_id'],
#                    right_on=['time_id', 'symbol_id'])

    # print(data)
    a = -0.00099
    b = 0.90162

    eps = 1e-10
#     model_pred = a + b * data['responder_6_lag_1']
    model_pred = a + b * X_test['responder_6_lag_1']
    test_preds = np.clip(model_pred,-5 + eps,5 - eps)
    model_pred.name = 'responder_6'
#     predictions = pd.concat((data['row_id'], test_preds), axis=1).fillna(0.0)
    predictions = pd.concat((X_test['row_id'], test_preds), axis=1).fillna(0.0)
    predictions.columns = ['row_id', 'responder_6']
#     predictions = predictions.loc[test.index]
#     print(type(predictions))

    # pred = data[_lag_1']]
    # print(pred)        
    # lags = lags.with_row_index('row_id')
    
#     a = -0.00099
#     b = 0.90162
#     print(type(test))
#     print(type(lags))
#     print(test.head())
#     print(lags.head())
    
    
#     predictions = lags.select(
#         'row_id',
#         (pl.lit(a) + pl.col('responder_6_lag_1') * pl.lit(b)).alias('responder_6'),
#     )

#     predictions = pd.DataFrame(a + b * lags["responder_6_lag_1"])
#     predictions['row_id'] = range(len(predictions))
#     print(predictions)
#     print(type(predictions))



#     # Replace this section with your own predictions
#     predictions = test.select(
#         'row_id',
#         pl.lit(0.0).alias('responder_6'),
#     )

    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
#         print('Here')
#         print(predictions.columns)
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
#         print(type(predictions))
        raise TypeError('The predict function must return a DataFrame')
    # Confirm has as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions

In [3]:
# test_df = pd.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet/date_id=0/part-0.parquet')
# test_df.head()

# print(len(test_df))


In [4]:
# lags_df = pl.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet/date_id=0/part-0.parquet')
# lags_df.head()
# print(type(lags_df))
# # lags_df = lags_df.with_row_count('row_id')
# print(len(lags_df))



In [5]:
# lags = lags_df.group_by(["date_id", "symbol_id"], maintain_order=True).last() # pick up last record of previous date
# print(lags_df)
# # test.join(lags, on=["date_id", "symbol_id"],  how="left")
# print(lags)

In [6]:
# data = pd.merge(left=test_df, right=lags_df, how='left', left_on=['date_id', 'time_id', 'symbol_id'],
#                right_on=['date_id', 'time_id', 'symbol_id'])

# # print(data)
# a = -0.00099
# b = 0.90162

# pred =  a + b * data['responder_6_lag_1']
# pred.columns = ['row_id', 'responder_6']
# # pred = data[_lag_1']]
# print(pred)

In [7]:
# test_df = pl.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet/date_id=0/part-0.parquet')
# lags_df = pl.read_parquet('/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet/date_id=0/part-0.parquet')
# y_pred = predict(test_df, lags_df)
# print(y_pred)

In [8]:
# import pandas as pd
# # Initialize a list to hold samples from each file
# samples = []
# # Load a sample from each file
# for i in range(10):
#     file_path = f"/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id={i}/part-0.parquet"
#     chunk = pd.read_parquet(file_path)
    
#     # Take a sample of the data (adjust sample size as needed)
#     sample_chunk = chunk.sample(n=100000, random_state=42)  # For example, 100 rows
#     samples.append(sample_chunk)
# # Concatenate all samples into one DataFrame if needed
# sample_df = pd.concat(samples, ignore_index=True)


When your notebook is run on the hidden test set, inference_server.serve must be called within 15 minutes of the notebook starting or the gateway will throw an error. If you need more than 15 minutes to load your model you can do so during the very first `predict` call, which does not have the usual 1 minute response deadline.

In [9]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )